This is a data prepeocessing code for the coal mine sagety monitering seismic data. The data sets were downloaded from the data mining competion: [AAIA'16 Data Mining Challenge: Predicting Dangerous Seismic Events in Active Coal Mines](https://knowledgepit.fedcsis.org/contest/view.php?id=112) 

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import h5py

There are five traing sets and their relevant labels, we need load them firstly.

In [3]:
df1=pd.read_csv('E:/数据/AAIA 16/trainingData/trainingData.csv',header=None)
df2=pd.read_csv('E:/数据/AAIA 16/additional_training_data_1/additional_training_data_1.csv',header=None)
df3=pd.read_csv('E:/数据/AAIA 16/additional_training_data_2/additional_training_data_2.csv',header=None)
df4=pd.read_csv('E:/数据/AAIA 16/additional_training_data_3/additional_training_data_3.csv',header=None)
df5=pd.read_csv('E:/数据/AAIA 16/additional_training_data_4/additional_training_data_4.csv',header=None)
test_data=pd.read_csv('E:/数据/AAIA 16/testData/testData.csv',header=None)
train_label1=pd.read_csv('E:/数据/AAIA 16/trainingData/trainingLabels.csv',header=None)
train_label2=pd.read_csv('E:/数据/AAIA 16/additional_training_data_1/additional_training_labels_1.csv',header=None)
train_label3=pd.read_csv('E:/数据/AAIA 16/additional_training_data_2/additional_training_labels_2.csv',header=None)
train_label4=pd.read_csv('E:/数据/AAIA 16/additional_training_data_3/additional_training_labels_3.csv',header=None)
train_label5=pd.read_csv('E:/数据/AAIA 16/additional_training_data_4/additional_training_labels_4.csv',header=None)
test_label=pd.read_csv('E:/数据/AAIA 16/evaluation_data/testLabels.csv',header=None)

Combine those sets.

In [4]:
train_data=pd.concat([df1,df2,df3,df4,df5])
train_label=pd.concat([train_label1,train_label2,train_label3,train_label4,train_label5])

The first row of the train_data is working site id, let's check it.

In [5]:
t=train_data.values
t=t[:,0]
pd.unique(t)

array([146, 264, 373, 437, 541, 725, 777, 149, 155, 575, 583, 765, 479,
       607, 793, 599], dtype=object)

Also, the test set.

In [6]:
t=test_data.values
t=t[:,0]
pd.unique(t)

array([765, 575, 777, 149, 725, 799, 490, 607, 689, 793, 599, 583, 470,
       155, 703, 479, 146, 508, 171, 641, 541], dtype=object)

There are also some levels expressed by "a,b,c,d" and "normal,warning".We replace "a,b,c,d" to "1,2,3,4", and "nomal, warning" to "0,1".

In [7]:
train_label=train_label.replace(['normal','warning'],[0,1])
test_label=test_label.replace(['normal','warning'],[0,1])

train_data = train_data.replace(['a', 'b', 'c', 'd'], [1, 2, 3, 4])
test_data = test_data.replace(['a', 'b', 'c', 'd'], [1, 2, 3, 4])

The values in the dataset are in different range, we need to do the standardization.

In [8]:
standard_index=[1,2,3,4,5,6,11]+list(range(133,253))+list(range(325,541))
replace_index=[7,8,9,10]

train_data=np.array(train_data)
train_label=np.array(train_label).reshape(-1)

test_data=np.array(test_data)
test_label=np.array(test_label).reshape(-1)

for i in standard_index:
    s = StandardScaler().fit(train_data[:, i])
    ss= StandardScaler().fit(test_data[:, i])
    train_data[:, i] = s.fit_transform(train_data[:, i])
    test_data[:, i] = ss.fit_transform(test_data[:, i])

C:\Users\13011\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\13011\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\13011\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single fea

Save the processed sets to h5 file

In [9]:
file = h5py.File('E:\\数据\\AAIA 16\\Pre_combined.h5', 'w')
file.create_dataset('train_label', data=train_label)
file.create_dataset('test_label', data=test_label)
file.create_dataset('train_data', data=train_data)
file.create_dataset('test_data', data=test_data)
file.close()

Let's check it.

In [10]:
file = h5py.File('E:\\数据\\AAIA 16\\Pre_combined.h5', 'r')
X_train = file['train_data'][:]
Y_train = file['train_label'][:]
X_test = file['test_data'][:]
Y_test = file['test_label'][:]
file.close()

In [11]:
X_train.shape

(133151, 541)

In [12]:
X_test.shape

(3860, 541)

In [14]:
Y_train.shape

(133151,)

In [15]:
Y_test.shape

(3860,)

Let's check there class distribution

In [17]:
from collections import Counter

Counter({0: 3664, 1: 196})

In [18]:
Counter(Y_train)

Counter({0: 130188, 1: 2963})

In [19]:
Counter(Y_test)

Counter({0: 3664, 1: 196})

The distribution are extremly imbalanced, the ratios are 43.94 and 18.7 respectively.